In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [3]:
FILE_ID = '1AhhYo_QWV9j6DCoisJJaZwJoB_b9sTQX'

In [4]:
!gdown 1AhhYo_QWV9j6DCoisJJaZwJoB_b9sTQX

Downloading...
From: https://drive.google.com/uc?id=1AhhYo_QWV9j6DCoisJJaZwJoB_b9sTQX
To: /content/archive.zip
100% 124M/124M [00:00<00:00, 255MB/s]


In [5]:
!unzip archive.zip

Archive:  archive.zip
replace Data/test/adenocarcinoma/000108 (3).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Data/test/adenocarcinoma/000108 (3).png  
replace Data/test/adenocarcinoma/000109 (2).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Data/test/adenocarcinoma/000109 (2).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Data/test/adenocarcinoma/000109 (2).png  
  inflating: Data/test/adenocarcinoma/000109 (4).png  
  inflating: Data/test/adenocarcinoma/000109 (5).png  
  inflating: Data/test/adenocarcinoma/000112 (2).png  
  inflating: Data/test/adenocarcinoma/000113 (7).png  
  inflating: Data/test/adenocarcinoma/000114 (5).png  
  inflating: Data/test/adenocarcinoma/000114.png  
  inflating: Data/test/adenocarcinoma/000115 (4).png  
  inflating: Data/test/adenocarcinoma/000115 (8).png  
  inflating: Data/test/adenocarcinoma/000115.png  
  inflating: Data/test/adenocarcinoma/000116 (5).png  
  inflating: Data/test/adenoca

In [6]:
!ls

archive.zip  Data  sample_data


In [7]:
!ls Data

test  train  valid


In [8]:
!ls Data/train

adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib     normal
large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa  squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa


In [9]:
!ls Data/valid

adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib     normal
large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa  squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa


In [10]:
train_data = image_dataset_from_directory('./Data/train', label_mode = 'categorical',
  labels = 'inferred',
  shuffle = False,
  seed = 42,
  image_size =(256, 256),
  batch_size = 32
)

val_data = image_dataset_from_directory('./Data/valid', label_mode = 'categorical',
  labels = 'inferred',
  shuffle = False,
  seed = 42,
  image_size =(256, 256),
  batch_size = 32
)

test_data = image_dataset_from_directory('./Data/test', label_mode = 'categorical',
  labels = 'inferred',
  shuffle = False,
  seed = 42,
  image_size =(256, 256),
  batch_size = 32
)

Found 613 files belonging to 4 classes.
Found 72 files belonging to 4 classes.
Found 315 files belonging to 4 classes.


In [11]:
train_data.take(10)

<_TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [12]:
def create_model(input_shape: tuple, output_shape: int) -> Model:
  input = Input(shape = input_shape)
  x = Conv2D(32, (3, 3), activation = 'relu')(input)
  x = Conv2D(64, (3, 3), activation = 'relu')(x)
  x = Conv2D(128, (3, 3), activation = 'relu')(x)
  x = Conv2D(64, (3, 3), activation = 'relu')(x)
  x = Conv2D(32, (3, 3), activation = 'relu')(x)
  x = Flatten()(x)
  output = Dense(units = output_shape, activation = 'softmax')(x)
  return Model(input, output)

In [17]:
model = create_model((256, 256, 3), 4)

In [18]:
model.compile(loss = 'categorical_crossentropy',
              optimizer=Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [19]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 conv2d_6 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 250, 250, 128)     73856     
                                                                 
 conv2d_8 (Conv2D)           (None, 248, 248, 64)      73792     
                                                                 
 conv2d_9 (Conv2D)           (None, 246, 246, 32)      18464     
                                                                 
 flatten_1 (Flatten)         (None, 1936512)           0   

In [ ]:
# Train the model
history = model.fit(
    train_data,
    epochs = 10,  # You can adjust the number of epochs
)

Epoch 1/10
 1/20 [>.............................] - ETA: 26:23 - loss: 3.2996 - accuracy: 0.1562